<font size="5"><b>Project Objectives:</b></font>

<font size="3">- With different datasets, create and manipulate databases with SQLite3;</font>

<font size="3">- Answer different kinds of business questions with queries;</font>

<font size="3">- Create DataFrame objects as results from queries for a better visualization, this also make possible to start drawing inferences utilizing Pandas DataFrame functions.</font>


<font size="3"><b>Imports for data and database manipulation</b></font>

In [1]:
import pandas as pd
import sqlite3 as s3

<font size="4">Initial settings</font>

In [2]:
# Removing limitations by setting it to None to display all columns.
pd.set_option('display.max_columns', None)
# Removing limitations by setting it to None to display all rows.
pd.set_option('display.max_rows', None) 

In [3]:
# Function to return all the queries into dataframes
def query_results(cursor):
    # Get the results from the query
    results = cursor.fetchall()

    # Get the column names from the cursor description
    column_names = [description[0] for description in cursor.description]

    # Create the DataFrame from the results and column names
    df_results = pd.DataFrame(results, columns=column_names)

    # Query results
    return df_results

<font size="5"><b>#001 Dataset - Automobile Data</b></font>

In [4]:
# Reading the .csv dataset - Creating the dataframe object
df_auto = pd.read_csv('automobile_data.csv')

# Creating conn type object and creating the empty database on the folder this jupyter is located
conn = s3.connect('automobile_data.sqlite')

# Creating cursor type object - this object can execute SQL queries
cursor = conn.cursor()

# Saving the automobile dataset on the newly created database
# Utilizing the dataframe as a source and the automobile_data.sqlite as a destination
df_auto.to_sql('automobile_data', conn, if_exists='replace', index=False)

df_auto.head()

,make,fuel_type,num_of_doors,body_style,drive_wheels,engine_location,wheel_base,length,width,height,curb_weight,engine_type,num_of_cylinders,engine_size,fuel_system,compression_ratio,horsepower,city_mpg,highway_mpg,price
0,alfa-romero,gas,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,9.0,111,21,27,13495
1,alfa-romero,gas,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,9.0,111,21,27,16500
2,alfa-romero,gas,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,9.0,154,19,26,16500
3,audi,gas,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,70.0,102,24,30,13950
4,audi,gas,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,8.0,115,18,22,17450


<font size="4">Business Question: Checking All Car <b>types</b> of Fuel and their <b>distribution</b></font>

In [5]:
# Checking for unique values on 'fuel_type' variable 
cursor.execute('''
SELECT
    DISTINCT fuel_type
FROM
    automobile_data
LIMIT
    1000
''')
query_results(cursor)

,fuel_type
0,gas
1,diesel


In [6]:
# Distribution
cursor.execute('''
SELECT
    fuel_type,
    COUNT(*) AS count
FROM
    automobile_data
GROUP BY
    fuel_type
''')
query_results(cursor)

,fuel_type,count
0,diesel,20
1,gas,183


<font size="4">Business question: What is the <b>average</b>, <b>minimum</b>, and <b>maximum</b> price of cars per <b>manufacturer?</b></font>

In [7]:
cursor.execute('''
SELECT
    make,
    AVG(price) AS avg_price,
    MIN(price) AS min_price,
    MAX(price) AS max_price
FROM
    automobile_data
GROUP BY
    make
ORDER BY
    min_price DESC;
''')
query_results(cursor)

,make,avg_price,min_price,max_price
0,jaguar,34600.000000,32250,36000
1,mercedes-benz,33647.000000,25552,45400
2,mercury,16503.000000,16503,16503
3,bmw,26118.750000,16430,41315
4,alfa-romero,15498.333333,13495,16500
5,volvo,18063.181818,12940,22625
6,peugot,15489.090909,11900,18150
7,saab,15223.333333,11850,18620
8,volkswagen,10077.500000,7775,13845
9,plymouth,7963.428571,5572,12764


<font size="4">Business question: What are the most <b>expensive</b> and  <b>cheapest</b> car models (make) in the dataset?</font>

In [8]:
cursor.execute('''
WITH MinMaxPrices AS (
    SELECT
        make,
        MIN(price) AS min_price,
        MAX(price) AS max_price
    FROM
        automobile_data
    WHERE
        price > 0
    GROUP BY
        make
)
SELECT
    (SELECT make FROM MinMaxPrices WHERE min_price = (SELECT MIN(min_price) FROM MinMaxPrices)) AS min_price_make,
    (SELECT min_price FROM MinMaxPrices WHERE min_price = (SELECT MIN(min_price) FROM MinMaxPrices)) AS min_price_value,
    (SELECT make FROM MinMaxPrices WHERE max_price = (SELECT MAX(max_price) FROM MinMaxPrices)) AS max_price_make,
    (SELECT max_price FROM MinMaxPrices WHERE max_price = (SELECT MAX(max_price) FROM MinMaxPrices)) AS max_price_value
''')
query_results(cursor)

,min_price_make,min_price_value,max_price_make,max_price_value
0,subaru,5118,mercedes-benz,45400


<font size="4"> Business Question: Identifying the <b>Shortest</b> and <b>Longest</b> Vehicles</font>

In [9]:
# Inspecting Minimum and Maximum values from 'length' variable.
cursor.execute('''
SELECT
    MIN(length) AS min_length,
    MAX(length) AS max_length
FROM
    automobile_data
''')
query_results(cursor)

,min_length,max_length
0,141.1,208.1


<font size="4"> Business Question: Identifying Cars with <b>Missing</b> Data</font>

In [10]:
# Finding missing data in all columns
cursor.execute('''
SELECT *
FROM automobile_data
WHERE
    (make IS NULL) OR
    (fuel_type IS NULL) OR
    (num_of_doors IS NULL) OR
    (body_style IS NULL) OR
    (drive_wheels IS NULL) OR
    (engine_location IS NULL) OR
    (wheel_base IS NULL) OR
    (length IS NULL) OR
    (width IS NULL) OR
    (height IS NULL) OR
    (curb_weight IS NULL) OR
    (engine_type IS NULL) OR
    (num_of_cylinders IS NULL) OR
    (engine_size IS NULL) OR
    (fuel_system IS NULL) OR
    (compression_ratio IS NULL) OR
    (horsepower IS NULL) OR
    (city_mpg IS NULL) OR
    (highway_mpg IS NULL) OR
    (price IS NULL);
''')
query_results(cursor)

,make,fuel_type,num_of_doors,body_style,drive_wheels,engine_location,wheel_base,length,width,height,curb_weight,engine_type,num_of_cylinders,engine_size,fuel_system,compression_ratio,horsepower,city_mpg,highway_mpg,price
0,dodge,gas,None,sedan,fwd,front,93.7,157.3,63.8,50.6,2191,ohc,four,98,mpfi,7.6,102,24,30,8558
1,mazda,diesel,None,sedan,fwd,front,98.8,177.8,66.5,55.5,2443,ohc,four,122,idi,22.7,64,36,42,10795


<font size="3">Updating the missing data with correct information</font>

In [11]:
# Filling missing data for 'dodge' car
cursor.execute('''
UPDATE
  automobile_data
SET
  num_of_doors = 'four'
WHERE
    make = 'dodge'
    AND fuel_type = 'gas'
    AND body_style = 'sedan';
''')

# Filling missing data for 'mazda' car
cursor.execute('''
UPDATE
    automobile_data
SET
    num_of_doors = 'four'
WHERE
    make = 'mazda'
    AND fuel_type = 'diesel'
    AND body_style = 'sedan';
''')

In [12]:
# Checking if there is still missing data
cursor.execute('''
SELECT
    *
FROM
    automobile_data
WHERE
    num_of_doors IS NULL;
''')
query_results(cursor)

,make,fuel_type,num_of_doors,body_style,drive_wheels,engine_location,wheel_base,length,width,height,curb_weight,engine_type,num_of_cylinders,engine_size,fuel_system,compression_ratio,horsepower,city_mpg,highway_mpg,price


<font size="3">Result Analysis : Zero results returned, all the missing data were sucessufully replaced from this variable</font>

<font size="4"> Business Question: Finding <b>Cylinder Variations</b> in Vehicles</font>

In [13]:
# Finding what values exists in a column
cursor.execute('''
SELECT
    DISTINCT num_of_cylinders
FROM
    automobile_data
''')
query_results(cursor)

,num_of_cylinders
0,four
1,six
2,five
3,three
4,twelve
5,two
6,tow
7,eight


<font size="3">Updating the mispelled word in data with correct information</font>

In [14]:
# Fixing a typo/misspelled word -'tow' to 'two'
cursor.execute('''
UPDATE
  automobile_data
SET
  num_of_cylinders = 'two'
WHERE
   num_of_cylinders = 'tow';
''')

In [15]:
# Checking if the misspelled word was replaced
cursor.execute('''
SELECT
    DISTINCT num_of_cylinders
FROM
    automobile_data
''')
query_results(cursor)

,num_of_cylinders
0,four
1,six
2,five
3,three
4,twelve
5,two
6,eight


Result Analysis : No 'tow' word found, all of the mispelled data were sucessufully replaced from this variable.

<font size="4"> Business Question: Finding <b>Minimum</b> and <b>Maximum</b> Compression Ratio Values </font>

In [16]:
# Checking min and max values for compression_ratio 
cursor.execute('''
SELECT
    MIN(compression_ratio) AS min_compression_ratio,
    MAX(compression_ratio) AS max_compression_ratio
FROM
    automobile_data;
''')
query_results(cursor)

,min_compression_ratio,max_compression_ratio
0,7.0,70.0


<font size="3">Updating the wrong data ranges with correct information</font>

In [17]:
# Checking if there is other wrong values, correct range: 7 to 23
cursor.execute('''
SELECT
    MIN(compression_ratio) AS min_compression_ratio,
    MAX(compression_ratio) AS max_compression_ratio
FROM
    automobile_data
WHERE
    compression_ratio <> 70;
''')
query_results(cursor)

,min_compression_ratio,max_compression_ratio
0,7.0,23.0


In [18]:
# Counting how many observations have compression_ratio = 70
cursor.execute('''
SELECT
    COUNT(*) AS rows_to_delete
FROM
    automobile_data
WHERE
    compression_ratio = 70;
''')
query_results(cursor)

,rows_to_delete
0,1


In [19]:
# Deleting the only one resulting row
cursor.execute('''
DELETE FROM
    automobile_data
WHERE
    compression_ratio = 70;
''')


In [20]:
# Counting how many observations have compression_ratio = 70
cursor.execute('''
SELECT
    COUNT(*) AS rows_to_delete
FROM
    automobile_data
WHERE
    compression_ratio = 70;
''')
query_results(cursor)

,rows_to_delete
0,0


<font size="3">Ensuring Consistencies: Using LENGTH function to check extra spaces on columns</font>

In [21]:
# Ensuring consistencies, using length to check extra spaces
cursor.execute('''
SELECT
    DISTINCT drive_wheels,
    LENGTH(drive_wheels) AS string_length
FROM
    automobile_data;
''')
query_results(cursor)

,drive_wheels,string_length
0,rwd,3
1,4wd,4
2,fwd,3
3,4wd,3


In [22]:
# Removing extra space with SQL TRIM function
cursor.execute('''
UPDATE
    automobile_data
SET
    drive_wheels = TRIM(drive_wheels)
WHERE TRUE;
''')

In [23]:
# Checking if the extra space was removed
cursor.execute('''
SELECT
    DISTINCT drive_wheels,
    LENGTH(drive_wheels) AS string_length
FROM
    automobile_data;
''')
query_results(cursor)

,drive_wheels,string_length
0,rwd,3
1,4wd,3
2,fwd,3


In [24]:
# Close connection with this database
cursor.close()
conn.close()

<font size="5"><b>#002 Dataset - Lauren Store Dataset</b></font>

In [25]:
df_lauren = pd.read_csv('Lauren-s-Furniture-Store-Transaction-Table.csv')
conn = s3.connect('lauren-store.sqlite')
cursor = conn.cursor()

df_lauren.to_sql('lauren_store_db', conn, if_exists='replace', index=False)
print(df_lauren.info())
df_lauren.head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            29 non-null     object 
 1   transaction_id  29 non-null     int64  
 2   customer_id     29 non-null     int64  
 3   product         27 non-null     object 
 4   product_code    29 non-null     object 
 5   product_color   29 non-null     object 
 6   product_price   29 non-null     object 
 7   purchase_size   29 non-null     int64  
 8   purchase_price  29 non-null     float64
 9   revenue         29 non-null     object 
dtypes: float64(1), int64(3), object(6)
memory usage: 2.4+ KB
None


,date,transaction_id,customer_id,product,product_code,product_color,product_price,purchase_size,purchase_price,revenue
0,2020-08-29 0:00:00,9900,1980,fan,SKU83503,brass,$13.99,2,27.98,$27.98


<font size="4"> Business Question: Finding the <b>Most Purchased</b> Product </font>

In [26]:
cursor.execute('''
SELECT
    product,
    SUM(purchase_size) AS total_purchases
FROM
    lauren_store_db
GROUP BY
    product
ORDER BY
    total_purchases DESC
LIMIT 1;
''')
query_results(cursor)

,product,total_purchases
0,rug,9


<font size="4"> Business Question: Identifying <b>Unique</b> Products and Their Corresponding <b>Colors</b> </font>

In [27]:
cursor.execute('''
SELECT DISTINCT
    product,
    product_color
FROM
    lauren_store_db;
''')
query_results(cursor)

,product,product_color
0,fan,brass
1,fan,white
2,fan,black
3,lamp,brass
4,bed,brown
5,couch,grey
6,couch,blue
7,couch,purple
8,couch,white
9,couch,brown


<font size="4"> Business Question: Calculating the <b>Total Revenue</b> for the Store </font>
* On this case the revenue value needed to be formatted without the dollar sign ($)

In [28]:
cursor.execute('''
SELECT
    SUM(CAST(REPLACE(revenue, '$', '') AS FLOAT)) AS total_revenue
FROM
    lauren_store_db;
''')
query_results(cursor)

,total_revenue
0,3682.62


<font size="4"> Business Question: Analyzing <b>Revenue Distribution</b> across Different <b>Products</b> </font>

In [29]:
cursor.execute('''
SELECT
    product,
    SUM(CAST(REPLACE(revenue, '$', '') AS FLOAT)) AS total_revenue
FROM
    lauren_store_db
GROUP BY
    product
ORDER BY
    total_revenue DESC;
''')
query_results(cursor)

,product,total_revenue
0,rug,808.65
1,bed,799.99
2,None,699.94
3,desk,509.85
4,chair,234.50
5,lamp,229.95
6,mirror,199.95
7,fan,111.92
8,bookcase,58.89
9,vase,19.98


<font size="4"> Business Question: Determining the <b>Ten most Customers</b> with the Highest <b>Total Purchase Price</b> </font>

In [30]:
cursor.execute('''
SELECT
    customer_id,
    SUM(purchase_price) AS total_purchase_price
FROM
    lauren_store_db
GROUP BY
    customer_id
ORDER BY
    total_purchase_price DESC
LIMIT 10;
''')
query_results(cursor)

,customer_id,total_purchase_price
0,1980,1027.98
1,9886,1000.00
2,9815,1000.00
3,7571,1000.00
4,4687,1000.00
5,4524,1000.00
6,3821,1000.00
7,1928,1000.00
8,335,1000.00
9,2512,799.99


<font size="4"> Business Question: Finding the <b>Busiest Month</b> for Transactions </font>

In [31]:
cursor.execute('''
SELECT
    SUBSTR(date, 1, 7) AS transaction_month,
    COUNT(*) AS transaction_count
FROM
    lauren_store_db
GROUP BY
    transaction_month
ORDER BY
    transaction_count DESC
LIMIT 1;
''')
query_results(cursor)

,transaction_month,transaction_count
0,2020-12,4


<font size="4"> Business Question: Identifying the <b>Transaction with the Highest Purchase Price</b> </font>

In [32]:
cursor.execute('''
SELECT
    transaction_id,
    MAX(purchase_price) AS highest_purchase_price
FROM
    lauren_store_db
GROUP BY
    transaction_id
ORDER BY
    highest_purchase_price DESC
LIMIT 1;
''')
query_results(cursor)

,transaction_id,highest_purchase_price
0,49430,1000.0


<font size="4"> Business Question: Analyzing the <b>Number of Purchases</b> for Each <b>Product</b> </font>


In [33]:
cursor.execute('''
SELECT
    product,
    COUNT(*) AS purchase_count
FROM
    lauren_store_db
GROUP BY
    product
ORDER BY
    purchase_count DESC;
''')
query_results(cursor)

,product,purchase_count
0,couch,9
1,fan,5
2,rug,4
3,desk,3
4,None,2
5,vase,1
6,mirror,1
7,lamp,1
8,chair,1
9,bookcase,1


<font size="4"> Business Question: Calculating the <b>Average Purchase Price</b> for Different <b>Colors</b> of Products </font>

In [34]:
cursor.execute('''
SELECT
    product_color,
    AVG(purchase_price) AS avg_purchase_price
FROM
    lauren_store_db
GROUP BY
    product_color
ORDER BY
    avg_purchase_price DESC;
''')
query_results(cursor)

,product_color,avg_purchase_price
0,purple,1000.000000
1,blue,1000.000000
2,grey,594.923333
3,white,545.985000
4,brown,439.756000
5,black,416.163333
6,beige,313.016667
7,brass,88.971000
8,green,19.980000


<font size="4"> Business Question: Finding the <b>Most Popular</b> Products in Terms of <b>Purchase Quantity</b> </font>

In [35]:
cursor.execute('''
SELECT
    product,
    SUM(purchase_size) AS total_purchase_quantity
FROM
    lauren_store_db
GROUP BY
    product
ORDER BY
    total_purchase_quantity DESC;
''')
query_results(cursor)

,product,total_purchase_quantity
0,rug,9
1,couch,9
2,fan,8
3,lamp,5
4,desk,3
5,vase,2
6,None,2
7,mirror,1
8,chair,1
9,bookcase,1


In [36]:
df_lauren.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            29 non-null     object 
 1   transaction_id  29 non-null     int64  
 2   customer_id     29 non-null     int64  
 3   product         27 non-null     object 
 4   product_code    29 non-null     object 
 5   product_color   29 non-null     object 
 6   product_price   29 non-null     object 
 7   purchase_size   29 non-null     int64  
 8   purchase_price  29 non-null     float64
 9   revenue         29 non-null     object 
dtypes: float64(1), int64(3), object(6)
memory usage: 2.4+ KB


<font size="4"> Business Question: <b>Ten most</b> Transaction <b>Prices</b></font>

In [37]:
# Sort all purchases by purchase_price in descending order
cursor.execute('''
SELECT
    purchase_price
FROM
    lauren_store_db
ORDER BY
    purchase_price DESC;
''')
df_results = query_results(cursor)

# Query results
df_results.head(11)

,purchase_price
0,1000.00
1,1000.00
2,1000.00
3,1000.00
4,1000.00
5,1000.00
6,1000.00
7,1000.00
8,1000.00
9,799.99


In [38]:
print(df_lauren.info())
df_lauren.head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            29 non-null     object 
 1   transaction_id  29 non-null     int64  
 2   customer_id     29 non-null     int64  
 3   product         27 non-null     object 
 4   product_code    29 non-null     object 
 5   product_color   29 non-null     object 
 6   product_price   29 non-null     object 
 7   purchase_size   29 non-null     int64  
 8   purchase_price  29 non-null     float64
 9   revenue         29 non-null     object 
dtypes: float64(1), int64(3), object(6)
memory usage: 2.4+ KB
None


,date,transaction_id,customer_id,product,product_code,product_color,product_price,purchase_size,purchase_price,revenue
0,2020-08-29 0:00:00,9900,1980,fan,SKU83503,brass,$13.99,2,27.98,$27.98


<font size="4"><b>Typecasting - converting data types with pandas</b></font>

In [39]:
from sqlalchemy import create_engine
# Função para remover caracteres e converter para float
def convert_to_float(value):
    if isinstance(value, str):
        value = value.replace('$', '').replace(',', '')  # Remover o caractere $ e a vírgula
    return float(value)

# Converter as colunas para float
df_lauren['product_price'] = df_lauren['product_price'].apply(convert_to_float)
df_lauren['purchase_price'] = df_lauren['purchase_price'].apply(convert_to_float)
df_lauren['revenue'] = df_lauren['revenue'].apply(convert_to_float)

# Salvar os dados no banco de dados SQLite3
engine = create_engine('sqlite:///lauren-store.sqlite')
df_lauren.to_sql('lauren-store', engine, if_exists='replace', index=False)

# Dataframe with data types changed
df_lauren.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            29 non-null     object 
 1   transaction_id  29 non-null     int64  
 2   customer_id     29 non-null     int64  
 3   product         27 non-null     object 
 4   product_code    29 non-null     object 
 5   product_color   29 non-null     object 
 6   product_price   29 non-null     float64
 7   purchase_size   29 non-null     int64  
 8   purchase_price  29 non-null     float64
 9   revenue         29 non-null     float64
dtypes: float64(3), int64(3), object(4)
memory usage: 2.4+ KB


<font size="4"><b> Typecasting - converting data types with SQL</b></font>

In [40]:
# Typecasting returning values with converted datatypes
cursor.execute('''
SELECT
    CAST(purchase_price AS FLOAT) AS purchase_price
FROM
    lauren_store_db
ORDER BY
    CAST(purchase_price AS FLOAT) DESC;    
''')
query_results(cursor)
df_results.head(3)

,purchase_price
0,1000.0
1,1000.0
2,1000.0


In [41]:
# Changing the Data Type of the 'purchase_price' Column to Permanent FLOAT, Altering the Database Itself
# Excluding the table to be able to re-run this cell for any potential code changes
cursor.execute('''
DROP TABLE IF EXISTS 
    "lauren-store";
''')

# Renomear a tabela atual
cursor.execute('''
ALTER TABLE
    "lauren_store_db"
RENAME TO
    "lauren-store_old";
''')

# Creating a new table with the updated schema
cursor.execute('''
CREATE TABLE
    "lauren_store_db"(
        date DATETIME,
        transaction_id INTEGER,
        customer_id INTEGER,
        product STRING,
        product_code STRING,
        product_color STRING,
        product_price FLOAT,
        purchase_size INTEGER,
        purchase_price FLOAT,
        revenue FLOAT);
''') 

# Copying data from the old table to the newly created table
cursor.execute('''
INSERT INTO
    "lauren_store_db"
SELECT 
    date,
    transaction_id,
    customer_id,
    product,
    product_code,
    product_color,
    CAST(product_price AS FLOAT),
    purchase_size,
    CAST(purchase_price AS FLOAT),
    CAST(revenue AS FLOAT) FROM "lauren-store_old";
''')

# Dropping the old table
cursor.execute('''
DROP TABLE
    "lauren-store_old";
''')

# Committing all the changes
conn.commit()

<font size="4"><b>SQLALCHEMY Library</b></font>

<font size="3">- Utilizing a schema with sqlalchemy lib and oriented-object programming to create the database</font>

In [42]:
from datetime import datetime
from sqlalchemy import create_engine, Column, Integer, String, Float, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# Defining the Schema using SQLAlchemy
Base = declarative_base()

class Transaction(Base):
    __tablename__ = 'lauren_store_db'
    id = Column(Integer, primary_key=True)
    date = Column(DateTime)
    transaction_id = Column(Integer)
    customer_id = Column(Integer)
    product = Column(String)
    product_code = Column(String)
    product_color = Column(String)
    product_price = Column(Float)  
    purchase_size = Column(Integer)
    purchase_price = Column(Float)  
    revenue = Column(Float) 

# Create the database and the table
engine = create_engine('sqlite:///lauren-store.sqlite')  # Alterado o nome do banco de dados
Base.metadata.create_all(engine)

# Creating the session to interact with the database
Session = sessionmaker(bind=engine)
session = Session()

# Creating a test DataFrame to avoid modifying the main df_lauren
df_test = df_lauren.copy()


# Inserir os dados no banco de dados
for _, row in df_test.iterrows():
    # Criando uma instância de datetime a partir da string de data
    date_obj = datetime.strptime(row['date'], '%Y-%m-%d %H:%M:%S')
    transaction = Transaction(
        date=date_obj,
        transaction_id=row['transaction_id'],
        customer_id=row['customer_id'],
        product=row['product'],
        product_code=row['product_code'],
        product_color=row['product_color'],
        product_price=row['product_price'],
        purchase_size=row['purchase_size'],
        purchase_price=row['purchase_price'],
        revenue=row['revenue']
    )
    session.add(transaction)

# Confirmar as alterações e fechar a sessão
session.commit()
session.close()

In [43]:
print(df_test.info())
df_test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            29 non-null     object 
 1   transaction_id  29 non-null     int64  
 2   customer_id     29 non-null     int64  
 3   product         27 non-null     object 
 4   product_code    29 non-null     object 
 5   product_color   29 non-null     object 
 6   product_price   29 non-null     float64
 7   purchase_size   29 non-null     int64  
 8   purchase_price  29 non-null     float64
 9   revenue         29 non-null     float64
dtypes: float64(3), int64(3), object(4)
memory usage: 2.4+ KB
None


,date,transaction_id,customer_id,product,product_code,product_color,product_price,purchase_size,purchase_price,revenue
0,2020-08-29 0:00:00,9900,1980,fan,SKU83503,brass,13.99,2,27.98,27.98
1,2020-05-01 0:00:00,12315,2463,fan,SKU83503,brass,13.99,2,27.98,27.98
2,2020-12-12 0:00:00,9890,1978,fan,SKU83503,white,13.99,1,13.99,13.99
3,2020-02-16 0:00:00,46915,9383,fan,SKU83503,black,13.99,1,13.99,13.99
4,2020-12-28 0:00:00,44700,8940,fan,SKU83503,brass,13.99,2,27.98,27.98


<font size="3">Extracting Date and Purchase Price: December 2020</font>


In [44]:
# Using CAST function with SUBSTR to extract and concatenate year, month, and day
cursor.execute('''
SELECT
    CAST(SUBSTR(date, 1, 4) || '-' || SUBSTR(date, 6, 2) || '-' || SUBSTR(date, 9, 2) AS DATE) AS date_only,
    purchase_price    
FROM
    lauren_store_db
WHERE
    date BETWEEN '2020-12-01' AND '2020-12-31';
''')
df_results = query_results(cursor)
df_results.head(3)

,date_only,purchase_price
0,2020,13.990
1,2020,27.980
2,2020,160.965


In [45]:
# Query to return date as string in "YYYY-MM-DD" format
cursor.execute('''
SELECT
    SUBSTR(date, 1, 10) AS date_only,
    purchase_price    
FROM
    lauren_store_db
WHERE
    date BETWEEN '2020-12-01' AND '2020-12-31';
''')
df_results = query_results(cursor)
df_results.head(3)

,date_only,purchase_price
0,2020-12-12,13.990
1,2020-12-28,27.980
2,2020-12-28,160.965


<font size="3">Concatenating strings, new product names can be used as unique keys</font>

In [46]:
# CONCAT(product_code, product_color) AS new_product_code - CONCAT not supported on sqlite3

# Using concatenate operator ||
cursor.execute('''
SELECT
    product_code || product_color AS new_product_code
FROM
    lauren_store_db
WHERE
    product = 'couch';
''')
df_results = query_results(cursor)
df_results.head(3)

,new_product_code
0,SKU31871grey
1,SKU31871grey
2,SKU31871grey


<font size="5"><b>#003 Dataset - Party Planet Dataset</b></font>
* Manipulate data with pandas and ensure consistencies on the database

In [47]:
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None)
df_planet = pd.read_excel('Jeffs-Party-Planet-Data-for-Cleaning.xlsx')
conn = s3.connect('jeff_planet.sqlite')
cursor = conn.cursor()

df_planet.to_sql('jeff_planet', conn, if_exists='replace', index=False)

print(df_planet.info())
df_planet.head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Product                34 non-null     object 
 1   Purchase Orders        34 non-null     int64  
 2   Suppliers              34 non-null     object 
 3   Price                  34 non-null     float64
 4   Expenditure per piece  34 non-null     float64
 5   Revenue                34 non-null     float64
 6   Total expenditure      34 non-null     float64
 7   Profit                 34 non-null     float64
dtypes: float64(5), int64(1), object(2)
memory usage: 2.3+ KB
None


,Product,Purchase Orders,Suppliers,Price,Expenditure per piece,Revenue,Total expenditure,Profit
0,Balloons (100 count),776,Sparklefest Ltd.,16.94,11.858,13145.44,9201.808,3943.632


<font size="4">Finding <b>unique, mispelling, invalid characters</b> on the dataset</font>

In [48]:
# Create a DataFrame to store the validation results
data = []

for column in df_planet.columns:
    if column in ["Purchase Orders", "Expenditure per piece", "Revenue", "Total expenditure", "Profit"]:
        for value in df_planet[column]:
            if any(char.isalpha() for char in str(value)):
                checks = ["Contains invalid characters"]
                data.append([column, value, ', '.join(checks)])

df_invalid = pd.DataFrame(data, columns=["Column", "Value", "Checks"])

# Display the DataFrame
df_invalid

,Column,Value,Checks


<font size="3">No results returned meaning no invalid characters were found on <b>quantitative</b> columns</font>

<font size="4"><b>Standardizing</b> Supplier Names</font>
* Updating 'Inflatibles Plos' to 'Inflatibles Plus' - Mispelled word
* Checking if there are still occurrences of this word in the created database.
* Updating the 'jeff_planet' table, setting 'Suppliers' column vaulues to 'Not Informed' where it is NULL

In [49]:
cursor.execute('''
UPDATE 
    jeff_planet
SET 
    Suppliers = 'Inflatibles Plus'
WHERE 
    Suppliers = 'Inflatibles Plos'
''')

cursor.execute('''
SELECT
    CASE
        WHEN Suppliers = 'Inflatibles Plos' THEN 'Inflatibles Plus'
        END AS cleaned_name
FROM
    jeff_planet
''')
df_results = query_results(cursor)
df_results.head(3)

,cleaned_name
0,None
1,None
2,None


In [50]:
# Execute the SQL command to count occurrences of the word 'Inflatibles Plos'
cursor.execute('''
SELECT
    COUNT(*) AS count_inflatibles_plos
FROM
    jeff_planet
WHERE
    Suppliers = 'Inflatibles Plos'
''')

# Retrieve the query result
result = cursor.fetchone()

# Extract the number of 'Inflatibles Plos' occurrences
occurrence_count = result[0]

# Display the number of occurrences
print(f"Occurrence Count: {occurrence_count}")

Occurrence Count: 0


In [51]:
# Updating Missing Supplier Information
# Update the 'jeff_planet' table, setting 'Suppliers' to 'Not Informed' where it is NULL
cursor.execute('''
UPDATE 
    jeff_planet
SET 
    Suppliers = 'Not Informed'
WHERE
    Suppliers IS NULL
''')

# Committing changes to the database
conn.commit()

In [52]:
cursor.execute('''
SELECT
    COUNT(*) AS not_informed_count
FROM
    jeff_planet
WHERE
    Suppliers = 'Not Informed'
''')
df_results = query_results(cursor)
df_results.head(3)

,not_informed_count
0,0


<font size="5"><b>#004 Dataset - Cosmetic dataset</b></font>

In [53]:
cosmetic_df = pd.read_excel('Cosmetics Inc. - Data for Optimizing.xlsx')
conn = s3.connect('cosmetic_data.sqlite')
cursor = conn.cursor()

cosmetic_df.to_sql('cosmetic_data', conn, if_exists='replace', index=False)
print(cosmetic_df.info())
cosmetic_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Product Codes  30 non-null     object 
 1   Price          30 non-null     float64
 2   Client         30 non-null     object 
 3   Client Code    30 non-null     object 
 4   Orders         30 non-null     int64  
 5   Total          30 non-null     float64
dtypes: float64(2), int64(1), object(3)
memory usage: 1.5+ KB
None


,Product Codes,Price,Client,Client Code,Orders,Total
0,51993Masc,9.98,Candy's Beauty Supply,PINNC980,191,1906.18
1,49631Foun,14.49,Rockland's,ARLVA283,152,2202.48
2,42292Glos,6.74,Rudiger Pharmacy,CHEMD763,758,5108.92
3,86661Shad,5.71,Elizabethtown Supply,COLSC761,308,1758.68
4,49541Eyel,7.94,Rockland's,ARLVA425,50,397.00


In [54]:
# Splitting the "Product Codes" column into two new columns P_Number and P_Code 
cosmetic_df[['P_Number', 'P_Code']] = cosmetic_df['Product Codes'].str.extract('(\d+)([A-Za-z]+)')

# Dropping the original "Product Codes" column
cosmetic_df.drop('Product Codes', axis=1, inplace=True)

# Reordering the columns
cosmetic_df = cosmetic_df[['P_Number', 'P_Code'] + list(cosmetic_df.columns[:-2])]

# Display the modified DataFrame
cosmetic_df.sample(5)

,P_Number,P_Code,Price,Client,Client Code,Orders,Total
10,85021,Foun,11.75,Rudiger Pharmacy,CHEMD339,707,8307.25
7,86139,Lips,5.55,Candy's Beauty Supply,PINNC496,299,1659.45
0,51993,Masc,9.98,Candy's Beauty Supply,PINNC980,191,1906.18
5,58337,Foun,13.57,Candy's Beauty Supply,PINNC939,673,9132.61
15,64762,Foun,12.95,Rudiger Pharmacy,CHEMD913,355,4597.25


<font size="4"> Business Question: Find the<b>Ten most</b> Product <b>Revenues</b></font>

In [55]:
# Retrieve Product Revenue and Calculate Total Revenue for Each Product
cursor.execute('''
SELECT
    Price,
    Orders,
    Price * Orders AS Total_revenue
FROM
    cosmetic_data
ORDER BY
    Total_revenue DESC
LIMIT 10
''')
query_results(cursor)

,Price,Orders,Total_revenue
0,20.04,782,15671.28
1,13.13,972,12762.36
2,12.06,797,9611.82
3,11.05,850,9392.50
4,13.57,673,9132.61
5,11.75,707,8307.25
6,15.77,514,8105.78
7,11.22,621,6967.62
8,16.94,362,6132.28
9,10.07,602,6062.14


<font size="4">Business Question: Calculate the <b>Total Revenue</b> with <b>10%</b> Discount</font>

In [56]:
# Retrieve Total Revenue and Total Revenue with a 10% Discount
cursor.execute('''
SELECT
    SUM(Price * Orders) AS Total_revenue,
    SUM((Price * Orders) * 0.90) AS Total_revenue_10
FROM
    cosmetic_data
''')
query_results(cursor)

,Total_revenue,Total_revenue_10
0,146928.99,132236.091
